# 🐚🐧 Penguin and Abalone Classifier 🐧🐚
Using different machine learning algorithms and different data sets, this notebook attempts to predict the species of a penguin given its features and the sex of an abalone given its features.

### Imports and Declarations

In [2]:
# data analysis and manipulation
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# model
from sklearn.tree import DecisionTreeClassifier as DTree
from sklearn.neural_network import MLPClassifier as MLP


## Dataset

In [ ]:
# INSERT CODE HERE
# read data from file
penguin = pd.read_csv('penguins.csv')
abalone = pd.read_csv('abalone.csv')

## Preprocessing

In [ ]:
# INSERT CODE HERE

## Charting

In [ ]:
# INSERT CODE HERE

## Data Splitting

In [ ]:
# INSERT CODE HERE

## Base Decision Tree

In [ ]:
# INSERT CODE HERE

## Top Decision Tree

In [ ]:
# INSERT CODE HERE

## Base MLP

In [ ]:
# INSERT CODE HERE

## Top MLP

In [ ]:
# INSERT CODE HERE

## Output Statistics

In [ ]:
# INSERT CODE HERE